In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import re
import glob
from preprocess import *

In [2]:
def load_Image(path):
    image=cv2.imread(path)
    return image

In [3]:
# def load_Dataset(path):
#     images = []
#     sorted_files = sorted(glob.glob(path), key=lambda x: int(re.search(r'\d+', x).group()))
#     for img in sorted_files:
#         n= cv2.imread(img)
#         cv2.destroyAllWindows()
#         images.append(n)
#     print(len(images))
#     return images

In [4]:
# def threshold_image(image):
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     cv2.imshow('gray', gray_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#    # median_filter = cv2.medianBlur(gray_image, 5)
#     # Calculate the histogram
#     hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
#     # Get the total number of pixels in the image
#     total_pixels = gray_image.shape[0] * gray_image.shape[1]
#     # Calculate the ratio of black pixels to white pixels
#     black_ratio = sum(hist[:50]) / total_pixels
#     white_ratio = sum(hist[200:]) / total_pixels
    
#     # Set the threshold accordingly
#     if black_ratio > white_ratio:
#         ret, thresh = cv2.threshold(gray_image,0, 255, cv2.THRESH_BINARY_INV  + cv2.THRESH_OTSU) #50
#     else: 
#         ret, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)   #180
#     # check salt and pepper to rethreshold
#     contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#     min_contour_area = 50
#     filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
#     if abs(len(filtered_contours)-len(contours))<8000 :
#        return thresh
#     else:
#         median_filter = cv2.medianBlur(gray_image, 5)
#         if black_ratio > white_ratio:
#             ret, thresh = cv2.threshold(median_filter,0, 255, cv2.THRESH_BINARY_INV  + cv2.THRESH_OTSU) #50
#         else: 
#             ret, thresh = cv2.threshold(median_filter, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)   #180
#     return thresh

In [5]:
def denoise(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    # Get the total number of pixels in the image
    total_pixels = gray_image.shape[0] * gray_image.shape[1]
    # Calculate the ratio of black pixels to white pixels
    black_ratio = sum(hist[:50]) / total_pixels
    white_ratio = sum(hist[200:]) / total_pixels
    
    # Set the threshold accordingly
    if black_ratio > white_ratio:
        ret, thresh = cv2.threshold(gray_image,0, 255, cv2.THRESH_BINARY_INV  + cv2.THRESH_OTSU) #50
    else: 
        ret, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)   #180
    # check salt and pepper to rethreshold
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    min_contour_area = 50
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    if abs(len(filtered_contours)-len(contours))<8000 :
       return gray_image
    else:
        median_filter = cv2.medianBlur(gray_image, 5)
        return median_filter

In [6]:
def get_words_grey(binr,img,resize=False):
    process_on=binr.copy()
    dilated_image = cv2.dilate(process_on, np.ones((3, 3), np.uint8), iterations=1)
    contours, _ = cv2.findContours(dilated_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # Filter out the small contours
    min_contour_area = 50
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    # Sort the contours from left to right
    filtered_contours = sorted(filtered_contours, key=lambda cnt: cv2.boundingRect(cnt)[0])
    # cv2.drawContours(x, filtered_contours, -1, (0, 255, 0), 3)
    # cv2.imshow('contours', x)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cropped_images = []
    
    # Iterate through each contour and crop the image
    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (w*h >1000 ):
            cropped_image = img[y:y+h, x:x+w]
            if resize:
                cropped_image = cv2.resize(cropped_image, (64, 64))
            cropped_images.append(cropped_image)
    
    # Display the cropped images (optional)
    # for i, cropped_img in enumerate(cropped_images):
    #     cv2.imshow(f'Cropped Image {i}', cropped_img)
    #     cv2.waitKey(0)
    
    # cv2.destroyAllWindows()
    #print (f'Number of words: {len(cropped_images)}')
    return cropped_images    

In [7]:
# def assure_white_bg(image):
#     hist = cv2.calcHist([image], [0], None, [256], [0, 256])
#     total_pixels = image.shape[0] * image.shape[1]
#     # Calculate the ratio of black pixels to white pixels
#     black_ratio = sum(hist[:50]) / total_pixels
#     white_ratio = sum(hist[200:]) / total_pixels
#     thresh=image.copy()
#     if black_ratio > white_ratio:
#         ret, thresh = cv2.threshold(image,50, 255, cv2.THRESH_BINARY_INV)
#     return thresh


In [8]:
#test1=load_Dataset("../fonts-dataset/Lemonada/*.jpeg")
# test1=load_Image("../fonts-dataset/Lemonada/0.jpeg")
# print(len(test1))
# #for img in test1:
# img=threshold_image(test1)
# img=assure_white_bg(img)
# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
test1=load_Dataset("../fonts-dataset/Lemonada/*.jpeg")

832


In [11]:
for img in test1:
    img = threshold_image(img)
    img= assure_white_bg(img)
    words=get_words(img)
    for word in words:
        cv2.imshow("w", word)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 

In [10]:
for img in test1:
    #x=img.copy()
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img,resize=True)
    for word in words:
        cv2.imshow("Image", word)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    # print(img.shape)
    # cv2.imshow("Image", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()